In [ ]:
from fastcore.all import *
from fastai.vision.all import *
from fastxtend.audio.all import *
from fastxtend.vision.all import *
from dataclasses import dataclass, field

from torchinfo import summary

T = TypeVar('T')
Listified = Union[T, Iterable[T], MutableSequence[T], L, fastuple]

In [ ]:
class EarlyStoppingWithWarmup(TrackerCallback):
    "A `TrackerCallback` that terminates training when monitored quantity stops improving."
    order=TrackerCallback.order+3
    def __init__(self, 
        monitor='valid_loss', # value (usually loss or metric) being monitored.
        comp=None, # numpy comparison operator; np.less if monitor is loss, np.greater if monitor is metric.
        min_delta=0., # minimum delta between the last monitor value and the best monitor value.
        patience=1, # number of epochs to wait when training has not improved model.
        warmup=5,
        reset_on_fit=True, # before model fitting, reset value being monitored to -infinity (if monitor is metric) or +infinity (if monitor is loss).
    ):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta, reset_on_fit=reset_on_fit)
        self.patience = patience
        self.warmup = warmup

    def before_fit(self): 
        self.wait = 0
        self.warmup_wait = self.warmup
        super().before_fit()
    def after_epoch(self):
        "Compare the value monitored to its best score and maybe stop training."
        super().after_epoch()
        self.warmup_wait = max(0, self.warmup_wait - 1)
        if self.new_best: self.wait = 0
        else:
            if self.warmup_wait == 0:
                self.wait += 1
            if self.wait >= self.patience and self.warmup_wait == 0:
                print(f'No improvement since epoch {self.epoch - self.wait}: early stopping')
                raise CancelFitException()

def save_model(learn: Learner, name, pickle=False):
    timestr = time.strftime("%m-%d-%Y")
    file_name = timestr + "/" + name
    os.makedirs("models/" + timestr, exist_ok=True)
    saved = learn.save(file_name)
    learn.save("latest")
    torch.save(learn.model, "checkpoints/latest.pt", )
    if pickle:
        os.makedirs("checkpoints/" + timestr, exist_ok=True)
        learn.export("checkpoints/" + file_name + ".pkl")
        learn.export("checkpoints/latest.pkl")
    return saved, file_name

class SaveBestState(TrackerCallback):
    "A `TrackerCallback` that saves the model's best during training and loads it at the end."
    order = TrackerCallback.order+1
    def __init__(self, 
        monitor='valid_loss', # value (usually loss or metric) being monitored.
        comp=None, # numpy comparison operator; np.less if monitor is loss, np.greater if monitor is metric.
        min_delta=0., # minimum delta between the last monitor value and the best monitor value.
        fname='model', # model name to be used when saving model.
        every_epoch=False, # if true, save model after every epoch; else save only when model is better than existing best.
        at_end=False, # if true, save model when training ends; else load best model if there is only one saved model.
        with_opt=False, # if true, save optimizer state (if any available) when saving model. 
        reset_on_fit=True # before model fitting, reset value being monitored to -infinity (if monitor is metric) or +infinity (if monitor is loss).
    ):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta, reset_on_fit=reset_on_fit)
        assert not (every_epoch and at_end), "every_epoch and at_end cannot both be set to True"
        # keep track of file path for loggers
        self.last_saved_path = None
        self.last_saved_names = None
        store_attr('fname,every_epoch,at_end,with_opt')

    def _save(self, name): self.last_saved_path, self.last_saved_name = save_model(self.learn, name)

    def _load_last(self):  self.learn.load(f'{self.fname}', with_opt=self.with_opt)
    
    def after_epoch(self):
        "Compare the value monitored to its best score and save if best."
        if self.every_epoch:
            if (self.epoch%self.every_epoch) == 0: self._save(self.fname)
        else: #every improvement
            super().after_epoch()
            if self.new_best:
                print(f'Better model found at epoch {self.epoch} with {self.monitor} value: {self.best}.')
                self._save(f'{self.fname}')
                print(f'Saved Path: {self.last_saved_path} Saved Name: {self.last_saved_name}')

    def after_fit(self, **kwargs):
        "Load the best model."
        if self.at_end: self._save(f'{self.fname}')
        elif not self.every_epoch: self.learn.load(f'{self.last_saved_name}', with_opt=self.with_opt)

In [ ]:
def binary_acc(y_pred, y):
    y,y_pred = TensorBase(y), TensorBase(y_pred)
    
    threshold = .5
    
    classes = int(y.shape[2] / 3)
    
    binary_true = torch.stack([y[:, :, x*3] for x in range(classes)], axis=1)
    binary_pred = torch.stack([y_pred[:, :, x*3] for x in range(classes)], axis=1)
    
    binary_true = torch.greater_equal(binary_true, threshold)
    binary_pred = torch.greater_equal(binary_pred, threshold)
    
    acc = (binary_true == binary_pred).type(torch.float32)
    acc = torch.mean(acc, (-1, -2, -3))
    
    return acc

def YOHOLoss(y_pred, y, beta=1.0):
    #beta changes how much of the regression loss counts
    # (bs, 30, 9) tensor shape
    y,y_pred = TensorBase(y), TensorBase(y_pred)
    
    classes = int(y.shape[2] / 3)
    
    squared_diff = torch.square(y_pred - y) 
    
    stack = []
    for c in range(classes):
        ss0 = squared_diff[:, :, c*3] * 0 + 1 #all 1s for binary label squared loss filter (they all count)
        ss1 = y[:, :, c*3] * beta #acts as a filter for regression loss
        ss2 = y[:, :, c*3] * beta
        stack += [ss0, ss1, ss2]
    
    sss = torch.stack(stack, dim=2)
    squared_diff = torch.multiply(squared_diff, sss)
    
    err = torch.sum(squared_diff, (-1, -2)) #sum the error for each time step, all the time steps
    #err = torch.mean(err, -1) #mean along the time steps
    #err = torch.sqrt(err)
    err = torch.mean(err) #mean among batches, we want loss to be independent of batch size
    if not err:
        print("ERROR NAN")
        print(y)
        print(y_pred)
    return err

In [ ]:
@dataclass
class DatasetOptions:
    categories: int = 2
    clip_duration: int = 6
    time_steps: int = 30
    
@dataclass
class TrainStage:
    name: str
    epochs: int = 10
    warmup: int = 5
    patience: int = -1
    beta: float = 1.0
    freeze: bool = True
    find_lr: bool = True
    lr_suggestion: str = "steep"
    
    def __post_init__(self):
        if self.patience < 0:
            self.patience = self.epochs
        
@dataclass
class ModelConfig:
    name: str
    batch_size: int
    output_channels: int
    pretrain: bool = True
    stages: list[TrainStage] = field(default_factory=list)
    train_stage: int = field(default=0)
    frozen: bool = field(default=True)
    lr: float = field(default=1e-5)
    save_best: SaveBestState = None
    
    def can_train(self):
        return self.train_stage < len(self.stages) - 1
       
    def train(self, learner: Learner):
        if self.can_train():
            stage: TrainStage = self.stages[self.train_stage]
            if stage.freeze and not self.frozen:
                learner.freeze()
                self.frozen = True
            elif not stage.freeze and self.frozen:
                learner.unfreeze()
                self.frozen = False
                
            print(f'\n ======= {self.name} - {stage.name} |-| b={stage.beta},warmup={stage.warmup},patience={stage.patience},frozen={self.frozen} ======= ')
            
            loss_func = partial(YOHOLoss, beta=stage.beta)
            learner.loss_func = loss_func
            
            if stage.find_lr:
                print(" --- LR Find ---")
                lrs: namedtuple = learner.lr_find(suggest_funcs=(minimum, steep, valley, slide))
                lrs_dict: dict = lrs._asdict()
                print(f' --- Found LR(s) --> {lrs} --- ')
                print(f'{stage.lr_suggestion} Rate --> {lrs_dict[stage.lr_suggestion]}')
                self.lr = lrs_dict[stage.lr_suggestion]
                
            
            
            print(" --- Fit One Cycle ---")
            #Store save best callback so that the "best" is actually the best across the stages
            if self.save_best:
                save_best = self.save_best
            else:
                save_best = SaveBestState(fname=self.name + "." + stage.name, reset_on_fit=False)
                self.save_best = save_best 
                
            early_stop = EarlyStoppingWithWarmup(monitor="valid_loss", patience=stage.patience, warmup=stage.warmup, reset_on_fit=False)
            callbacks = [ShowGraphCallback(), save_best, early_stop]
            learner.fit_one_cycle(stage.epochs, self.lr, cbs=callbacks)
            
            self.train_stage += 1

In [ ]:
def create_fake_output(time_steps, categories):
    t = np.zeros((time_steps, categories*3))
    for c in range(categories):
        for r in range(t.shape[0]):
            t[r, c*3] = np.random.choice([0, 1])
            t[r, c*3 + 1] = np.random.uniform(low=0, high=1.0)
            t[r, c*3 + 2] = np.random.uniform(low=t[r, c*3 + 1], high=1.0)
        
    return [t]

def alter_output(output, alter_binary=.15, alter_times=0):
    altered = np.copy(output)[0]
    time_steps = output[0].shape[0]
    categories = int(output[0].shape[1] / 3)
    
    for c in range(categories):
        for r in range(time_steps):
            if np.random.rand() <= alter_binary:
                altered[r, c*3] = (1 - altered[r, c*3])
            if np.random.rand() <= alter_times:
                altered[r, c*3 + 1] = np.random.uniform(low=0, high=1.0)
                altered[r, c*3 + 2] = np.random.uniform(low=altered[r, c*3 + 1], high=1.0)
        
    return [altered]
    
def get_dls(ds, bs, ds_options: DatasetOptions):
    dls = DataBlock(
        blocks=(MelSpecBlock(sr=16000, hop_length=160, n_fft=2048), RegressionBlock(n_out=3*ds_options.categories*ds_options.time_steps)),
        get_x=ColReader('File'),
        get_y=ColReader(0),
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        item_tfms=RandomCropPad(ds_options.clip_duration, samples=16000*ds_options.clip_duration, padmode=AudioPadMode.Constant),
        batch_tfms=[AmplitudeToDB(top_db=80), FrequencyMasking(max_mask=.2), Volume(p=.75, gain_range=(-18, 6))]
    )
    return dls.dataloaders(ds, bs=bs, num_workers=num_cpus(), pin_memory=True, verbose=False)


In [ ]:
model_configs = {
    0: ModelConfig(
        "convnext_small.fb_in22k_ft_in1k", 
        batch_size=64,
        output_channels=768,
        pretrain=True,
        stages = [
            TrainStage("stage0-0", epochs=16, beta=1, freeze=True, lr_suggestion="valley"), 
            TrainStage("stage0-1", epochs=5, beta=1, freeze=True, lr_suggestion="minimum"), 
            
            TrainStage("stage1", epochs=50, patience=5, warmup=5, freeze=False, lr_suggestion="steep"),
            TrainStage("stage2", epochs=5, beta=2, freeze=True, find_lr=False), #regression focus
            TrainStage("stage3", epochs=5, beta=2e-1, freeze=True, find_lr=False), #binary focus
            TrainStage("stage4", epochs=5, beta=1, freeze=True, find_lr=False), #balanced focus
        ]
    ),
    1: ModelConfig(
        "convnextv2_base.fcmae_ft_in22k_in1k_384", 
        batch_size=64,
        output_channels=1024,
        pretrain=True,
        stages = [
            TrainStage("stage0-0", epochs=30, patience=10, warmup=5, beta=1, freeze=True, lr_suggestion="valley"), 
            TrainStage("stage0-1", epochs=20, patience=7, warmup=5, beta=1, freeze=True, lr_suggestion="valley"), 
            
            TrainStage("stage1", epochs=50, patience=5, warmup=5, freeze=False, lr_suggestion="minimum"),
            TrainStage("stage2", epochs=20, patience=5, warmup=0, beta=2, freeze=True, find_lr=False),
            TrainStage("stage2", epochs=20, patience=5, warmup=0, beta=1e-2, freeze=True, find_lr=False),
        ]
    ),
}

mod_cfg = model_configs[0]
ds_options = DatasetOptions(categories=2, clip_duration=6, time_steps=30)

In [ ]:
    
df = pd.read_csv("X:\ML\Datasets\koe\labels.csv")
df2 = df.groupby('File', sort=False).apply(np.array, include_groups=False).reset_index()#.apply(np.ravel).reset_index()#

dls = get_dls(df2, bs=mod_cfg.batch_size, ds_options=ds_options)

#output of the base_model will be 4 x 18 x 1024 = 4 x duration*3 x 1024
last_channels = mod_cfg.output_channels

head_def = []

# head_def = [
#     ([3, 3], 1, 1024),
#     ([3, 3], 1, 512),
#     ([3, 3], 1, 256),
#     ([3, 3], 1, 128),
#     #([3, 3], 1, 64),
# ]

c = last_channels
while c >= 128:
    head_def.append(([3, 3], 1, int(c)))
    c = c/2

mod_dict = OrderedDict()
for i in range(len(head_def)):
    l = head_def[i]
    
    mod_dict[str(i) + '-' + "conv2d_dw"] = nn.Conv2d(kernel_size=l[0], stride=l[1], in_channels=last_channels, out_channels=last_channels, groups=last_channels, padding="same") #depthwise
    mod_dict[str(i) + '-' + "batchnorm_0"] = batch_norm_0 = nn.BatchNorm2d(eps=1e-4, num_features=last_channels)
    mod_dict[str(i) + '-' + "relu_0"] = nn.ReLU()
    mod_dict[str(i) + '-' + "conv2d"] = nn.Conv2d(in_channels=batch_norm_0.num_features, out_channels=l[2],  kernel_size=[1, 1], stride=1, padding="same")
    mod_dict[str(i) + '-' + "batchnorm_1"] = nn.BatchNorm2d(eps=1e-4, num_features=l[2])
    mod_dict[str(i) + '-' + "relu_1"] = nn.ReLU()
    
    last_channels = l[2]

mod_dict["reshape"] = nn.Flatten(start_dim=1, end_dim=2) #reshape 64 x 4 x 18 into 256 x 18 etc
mod_dict["adaptive2dpool"] = nn.AdaptiveAvgPool2d(output_size=(256, ds_options.categories*3))
mod_dict["conv1d"] = nn.Conv1d(256, out_channels=ds_options.time_steps, kernel_size=1)
mod_dict["sigmoid"] = nn.Sigmoid()
custom_head = nn.Sequential(mod_dict)

learn = vision_learner(dls, mod_cfg.name, opt_func=adam(foreach=True), custom_head=custom_head, pretrained=mod_cfg.pretrain, loss_func=YOHOLoss, metrics=[binary_acc], n_in=1, pool=False, concat_pool=False).to_channelslast()

#learn.model
#learn.summary()

while mod_cfg.can_train():
    mod_cfg.train(learn)